## Web-search RAG chatbot

like perplexity

v0.1 - no conversation, just question and answer based on web search

process:
1) receive prompt
2) pass to LLM to create appropriate search terms for gathering data - start with just one
3) perform web search with that, get top results and scrape and store as doc - start with just one result
4) chunk data
5) embed chunks
6) store in vectordb
7) link retriever to vecdb
8) pass query with retriever to llm w/ langchain

## 1) Receive prompt

In [3]:
user_prompt = "how fast is an F1 car?"

## 2) Pass to LLM to create search term

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

system_prompt = (
    "You are an assistant for gathering data based on this prompt."
    "Use the following prompt to generate one web search query that would gather data which would be helpful in answering the prompt."
    "Your search query should be effective at finding the most relevant and useful data pertaining to the prompt."
    "Keep in mind that the current date is July 2025"
    "\n\n"
    "{input}"
)

search_query_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [8]:
search_query_gen_chain = {"input": RunnablePassthrough()} | search_query_prompt | llm | StrOutputParser()

In [9]:
import ast

search_query = search_query_gen_chain.invoke(user_prompt)
search_query = ast.literal_eval(search_query)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [10]:
search_query = search_query.replace(' ', '+')

In [11]:
search_query

'current+F1+car+top+speed+2025'

In [12]:
google_search_url = f"https://www.google.com/search?q={search_query}"

In [13]:
google_search_url

'https://www.google.com/search?q=current+F1+car+top+speed+2025'

In [17]:
import requests

search_response = requests.get(url=google_search_url)
search_response

<Response [200]>

In [23]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(search_response.text, "html.parser")
found_links = soup.find_all("a")

In [24]:
found_links

[<a href="/?sa=X&amp;ved=0ahUKEwjQj9qlysGOAxXQXGwGHQXCBB0QOwgC"><span class="V6gwVd">G</span><span class="iWkuvd">o</span><span class="cDrQ7">o</span><span class="V6gwVd">g</span><span class="ntlR9">l</span><span class="iWkuvd tJ3Myc">e</span></a>,
 <a href="/search?q=current+F1+car+top+speed+2025&amp;sca_esv=8d3052da52eb7266&amp;ie=UTF-8&amp;gbv=1&amp;sei=_bV3aJCdK9C5seMPhYST6AE">here</a>,
 <a class="eZt8xd" href="/search?q=current+F1+car+top+speed+2025&amp;sca_esv=8d3052da52eb7266&amp;ie=UTF-8&amp;tbm=isch&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwjQj9qlysGOAxXQXGwGHQXCBB0Q_AUIBigB">Images</a>,
 <a class="eZt8xd" href="/search?q=current+F1+car+top+speed+2025&amp;sca_esv=8d3052da52eb7266&amp;ie=UTF-8&amp;tbm=vid&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwjQj9qlysGOAxXQXGwGHQXCBB0Q_AUIBygC">Videos</a>,
 <a class="eZt8xd" href="/search?q=current+F1+car+top+speed+2025&amp;sca_esv=8d3052da52eb7266&amp;ie=UTF-8&amp;tbm=nws&amp;source=lnms&amp;sa=X&amp;ved=0ahUKEwjQj9qlysGOAxXQXGwGHQXCBB0Q_AUICC

In [28]:
from urllib.parse import urlparse, unquote

target_url = ''

for i in found_links:
    href = i['href']
    if href.startswith('/url?q='):
        # exclude irrelevant findings like maps etc.
        # toy version for now, replace with more robust process like blacklist text file later

        target_url = href.split('/url?q=')[1].split('&')[0]
        domain = urlparse(unquote(target_url)).netloc.lower()
        if domain.endswith('google.com'):
            continue  # not a real organic result
        
        break

print(target_url)

https://racingnews365.com/f1-car-top-speed


## Get page as text data